In [ ]:
!pip install nltk beautifulsoup4 wordfreq spacy matplotlib scikit-learn textblob spacytextblob gensim --quiet --quiet
!python -m spacy download en_core_web_lg
!python -m textblob.download_corpora

In [ ]:
from bs4 import BeautifulSoup

with open('topics-task3.xml', 'r') as f:
    data = f.read()

data = BeautifulSoup(data, "xml")
topics = [tags.findAll(text=True)[0] for tags in  data.findAll('title', text=True)]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import pylab
from wordfreq import zipf_frequency, word_frequency


def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_ + '\n' + str(zipf_frequency(str(node.orth_).lower(), 'en')), [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_ + '\n' + str(zipf_frequency(str(node.orth_).lower(), 'en'))


cluster_count = 2
km = KMeans(n_clusters=cluster_count)
nlp = spacy.load('en_core_web_lg')
cm = pylab.get_cmap('gist_rainbow')



# see following code for 5.5 zipf threshold
# docs = list(nlp.pipe((str(t) for t in topics)))
# tokens = [token.lemma_ for doc in docs for token in doc if token.pos_ != 'PUNCT']
# frequencies = np.array([zipf_frequency(t, 'en') for t in tokens])
# labels = km.fit(frequencies.reshape(-1, 1)).labels_
# plt.scatter(frequencies, len(frequencies) * [1], color=np.array([cm(1.*l/cluster_count) for l in labels]))
# plt.xlim(5.4, 5.6)
# plt.show()


for t in topics[:20]:
    t = str(t)
    doc = nlp(t)
    words = [(str(x), zipf_frequency(str(x), 'en'), x.pos_) for x in doc]
    words.sort(key= lambda w: w[1])
    frequencies = [zipf_frequency(str(x), 'en') for x in doc]
    frequencies = np.array([f for f in frequencies if f > 0])
    
    plt.scatter(frequencies, len(frequencies) * [1], color=[cm(0 if f < 5.5 else 0.5) for f in frequencies])
    plt.show()
    #print([s[0] for s in words if s[2] == 'NOUN'])
    print(words)
    [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [156]:
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [183]:
print(glove_vectors.similarity('ban', 'yes'))
print(glove_vectors.similarity('ban', 'no'))

0.34057117
0.5165411


In [186]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from gensim.models import Word2Vec

nlp = spacy.load('en_core_web_lg')
nlp.add_pipe('spacytextblob')

docs = list(nlp.pipe((str(t) for t in topics)))

for doc in docs:
    root = str(next(doc.sents).root.lemma_)
    root_stance = next(doc.sents).root._.blob.polarity
    positive = glove_vectors.similarity(root, 'yes') > glove_vectors.similarity(root, 'no')
    words = [str(x) for x in doc if zipf_frequency(str(x), 'en') < 5.6 and x.pos_ != 'PUNCT' and str(x) != root]
    print (doc, '->', root, root_stance, '|', ' '.join(words))

Do we need sex education in schools? -> need 0.0 | sex education schools
Should stem cell research be expanded? -> expand 0.0 | stem cell research expanded
Should blood donations be financially compensated? -> compensate 0.0 | blood donations financially compensated
Should suicide be a criminal offense? -> be 0.0 | suicide criminal offense
Should agricultural subsidies be reduced? -> reduce 0.0 | agricultural subsidies reduced
Should vigilantism be legal? -> be 0.0 | vigilantism legal
Are gender or racial quotas effective? -> be 0.0 | gender racial quotas effective
Should holders of public offices resign on bad approval ratings? -> resign 0.0 | holders public offices bad approval ratings
Should nuclear weapons be abolished? -> abolish 0.0 | nuclear weapons abolished
Should the press be subsidized? -> subsidize 0.0 | press subsidized
Is wind power the best alternative energy source? -> be 0.0 | wind power alternative energy source
Is utilitarianism morally acceptable? -> be 0.0 | utilit

In [ ]:
for t in topics:
    doc = nlp(str(t))
    print(list(doc.noun_chunks))

In [ ]:
import nltk
from nltk.tree import *


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk import pos_tag, word_tokenize, RegexpParser

tagged = pos_tag(word_tokenize(topics[2]))

#Extract all parts of speech from any text
chunker = RegexpParser("""
                    NP: {<DT>?<JJ>*<NN>} #To extract Noun Phrases
                    P: {<IN>}            #To extract Prepositions
                    V: {<V.*>}           #To extract Verbs
                    PP: {<p> <NP>}       #To extract Prepositional Phrases
                    VP: {<V> <NP|PP>*}   #To extract Verb Phrases
                    """)

# Print all parts of speech in above sentence
output = chunker.parse(tagged)

output.draw()